In [5]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.wcs import WCS
from astropy.table import Table
from astropy.coordinates import SkyCoord
from astropy import units as u
from photutils.detection import DAOStarFinder
from photutils.background import Background2D, MedianBackground
import os
import pandas as pd

# --- Set the working directory ---
working_directory = r"C:\Users\friesco\workstation\fr-p\studies\ASTRO716\data_excercise"

# --- Function to align images based on star coordinates from TSV file ---
def align_images(reference_image_path, image_paths, tsv_file):
    """
    Aligns images to a reference image based on star coordinates read from a TSV file.

    Args:
        reference_image_path (str): Path to the reference image.
        image_paths (list): List of paths to images to be aligned.
        tsv_file (str): Path to the TSV file containing star coordinates.

    Returns:
        list: A list of aligned image data arrays.
    """
    aligned_images = []

    # Load the reference image
    ref_hdu = fits.open(reference_image_path)
    ref_data = ref_hdu[0].data
    ref_wcs = WCS(ref_hdu[0].header)
    ref_hdu.close()

    # Read the TSV file, skipping lines with invalid data
    data = []
    with open(tsv_file, 'r') as f:
        for line in f:
            if line.startswith('#') or len(line.split('\t')) < 23:  # Skip comments and lines with not enough fields
                continue
            data.append(line.strip().split('\t'))

    # Convert to DataFrame
    columns = ['ID', '_r', '_RAJ2000', '_DEJ2000', 'RAJ2000', 'DEJ2000', 'objID', 'f_objID', 'Qual', 'Ns', 'Nd', 'gmag', 'e_gmag', 'gFlags', 'rmag', 'e_rmag', 'rFlags', 'imag', 'e_imag', 'iFlags', 'zmag', 'e_zmag', 'zFlags']
    df = pd.DataFrame(data, columns=columns)

    # Convert relevant columns to numeric types
    df['RAJ2000'] = pd.to_numeric(df['RAJ2000'], errors='coerce')
    df['DEJ2000'] = pd.to_numeric(df['DEJ2000'], errors='coerce')

    # Drop rows with NaN values (invalid coordinates)
    df.dropna(subset=['RAJ2000', 'DEJ2000'], inplace=True)

    # Extract reference star coordinates (first 3 rows)
    ref_star_data = df.head(3)
    ref_star_coords = {}
    for index, row in ref_star_data.iterrows():
        ref_star_coords[f'star{index + 1}'] = (row['RAJ2000'], row['DEJ2000'])

    for image_path in image_paths:
        # Load the image to be aligned
        hdu = fits.open(image_path)
        data = hdu[0].data
        wcs = WCS(hdu[0].header)

        # Convert reference star coordinates to pixel coordinates in the current image
        ref_pix_coords = {}
        for star_name, (ref_ra, ref_dec) in ref_star_coords.items():
            skycoord = SkyCoord(ra=ref_ra, dec=ref_dec, unit='deg')
            x, y = wcs.world_to_pixel(skycoord)
            ref_pix_coords[star_name] = (x, y)

        # Use the first star's coordinates for alignment
        if ref_pix_coords:  # Check if ref_pix_coords is not empty
            ref_pix_coord = ref_pix_coords['star1']
        else:
            print(f"Error: Could not get pixel coordinates for reference stars in {image_path}")
            hdu.close()
            continue

        # Find the closest star in the current image to the reference star
        min_dist = float('inf')
        closest_star_coord = None
        for index, row in df.iterrows():
            ra, dec = row['RAJ2000'], row['DEJ2000']
            skycoord = SkyCoord(ra=ra, dec=dec, unit='deg')
            x, y = wcs.world_to_pixel(skycoord)

            dist = np.sqrt((ref_pix_coord[0] - x)**2 + (ref_pix_coord[1] - y)**2)
            if dist < min_dist:
                min_dist = dist
                closest_star_coord = (x, y)

        # Calculate the transformation matrix (using a simple translation for now)
        dx = ref_pix_coord[0] - closest_star_coord[0]
        dy = ref_pix_coord[1] - closest_star_coord[1]

        # Shift the WCS of the current image
        wcs.wcs.crpix[0] += dx
        wcs.wcs.crpix[1] += dy

        # Create a new header with the updated WCS
        aligned_header = hdu[0].header.copy()
        aligned_header.update(wcs.to_header())

        # Create a new HDU with the aligned header
        aligned_hdu = fits.PrimaryHDU(data, header=aligned_header)

        # Append the aligned data to the list
        aligned_images.append(aligned_hdu)

        hdu.close()

    return aligned_images

# --- Main execution ---
if __name__ == "__main__":
    # Define the reference image (first image)
    reference_image_path = os.path.join(working_directory, "phot_00.fits")

    # Define the list of images to be aligned (excluding the reference image)
    image_paths = [os.path.join(working_directory, f"phot_{i:02d}.fits") for i in range(1, 18)]

    # Define the path to the TSV file
    tsv_file = os.path.join(working_directory, "psdr1_new.tsv")

    # Load the reference image (not used for alignment, only for plotting)
    ref_hdu = fits.open(reference_image_path)
    ref_data = ref_hdu[0].data
    ref_header = ref_hdu[0].header
    ref_hdu.close()

    # Align the images
    aligned_images = align_images(reference_image_path, image_paths, tsv_file)

    # Save or process the aligned images
    for i, aligned_hdu in enumerate(aligned_images):
        output_path = os.path.join(working_directory, f"aligned_{i+1:02d}.fits")
        aligned_hdu.writeto(output_path, overwrite=True)
        print(f"Saved aligned image to {output_path}")

KeyError: 'star1'